In [ ]:
import optuna

In [ ]:
class OptunaTrainer:
    def __init__(self, train_loader, val_loader):
        self.train_loader = train_loader
        self.val_loader = val_loader
        
    def train_model(self, learning_rate, optimizer):
        model = # model instance e.g ResNet(num_classes)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        criterion = nn.CrossEntropyLoss()
        
        for epoch in range(NUM_EPOCHS):
            for inputs, labels in self.train_loader:
                #get data to cuda if possible
                inputs = inputs.to(device, dtype = torch.float32)
                labels = labels.to(device, dtype = torch.long)
                
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs.squeeze(), labels)
                loss.backward()
                optimizer.step()
                
        ## test evaluation
        model.eval()
        num_correct = 0
        total_samples = 0
        
        with torch.no_grad():
            for inputs, labels in self.val_loader:
                outputs = model(inputs)
                
                _, predicted = torch.max(outputs, dim=1)
                num_correct += (predicted == labels).sum()
                total_samples += labels.size(0)
        test_accuracy = num_correct / total_samples
        return test_accuracy
    
    def objective(self, trial):
        ## suggest hyperparams
        learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-1)
        optimizer = trial.suggest_categorical("optimizer", ["SGD", "Adam"])
        
        test_accuracy = self.train_model(learning_rate, optimizer)
        return test_accuracy

In [ ]:
tuner = OptunaTrainer(train_dataloader, val_dataloader)
study = optuna.create_study(direction="maximize")
study.optimize(tuner.objective, n_trials=5)

print("number of finished trials:", len(study.trials))
print("Best trial:")

trial = study.best_trial
print(" value:  ", trial.value)

print(" params:  ")
for key, value in trial.params.items():
    print("   {}: {}".format(key, value))